In [ ]:
import pickle
from ruamel.yaml import YAML as ym
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
def load_old_style_pickle(pickle_path, config_path):
    yaml = ym()
    config = yaml.load(open(config_path))

    # Old data format has a series of pickles, each containing one experiment of data
    data = []
    with open(pickle_path, "rb") as f:
        while True:
            try:
                data.append(pickle.load(f))
            except EOFError:
                break
    
    # Convert to a pandas dataframe
    df = pd.DataFrame(data)
    df = df.drop_duplicates()
    
    df["duty_cycle"] = config["CHIRP"]["rx_duration"] / df["values"]
    df["error_rate"] = df["n_error_list"] / df["n_pulse_attempts"]

    return df

def load_new_style_pickle(pickle_path):
    with open(pickle_path, "rb") as f:
        data = pickle.load(f)
    
    config = data["config"]
    data.pop("config")

    # Convert to a pandas dataframe
    df = pd.DataFrame(data)
    if "values" in df.columns:
        df["pri"] = df["values"]
    
    df["duty_cycle"] = config["CHIRP"]["rx_duration"] / df["pri"]
    df["error_rate"] = df["n_error_list"] / df["n_pulse_attempts"]

    return df


In [ ]:
config = data[0]["config"]
# remove key from dictionary
data[0].pop("config")
pd.DataFrame(data[0])

In [ ]:
duty_cycle_pickles = [
    # (path to pickle, label, is it in the new data format?)
    ("/media/thomas/Extreme SSD/orca_paper_data_files/duty_cycle/x310/20231220_114225_error_code_late_command.pickle", "X310, 10 Gbit Ethernet, Laptop", False),
    ("/media/thomas/Extreme SSD/orca_paper_data_files/duty_cycle/b205/20240227_200503_error_code_late_command.pickle", "B205, USB 3, Laptop", False),
    ("/media/thomas/Extreme SSD/orca_paper_data_files/duty_cycle/x310/20231010_044215_error_code_late_command.pickle", "X310, 1 Gbit Ethernet, Laptop", True),
    ("/media/thomas/Extreme SSD/orca_paper_data_files/duty_cycle/b205/20240227_142644_error_code_late_command.pickle", "B205, USB 3, Pi 4", False),
]

config_path = "/media/thomas/Extreme SSD/orca_paper_data_files/duty_cycle/b205/duty_cycle_b205.yaml"

duty_cycle_dfs = {}

for path, label, is_new_format in duty_cycle_pickles:
    if is_new_format:
        df = load_new_style_pickle(path)
    else:
        df = load_old_style_pickle(path, config_path)
    duty_cycle_dfs[label] = df

In [ ]:
matplotlib.rcParams.update({
        'font.size': 16,
        'legend.fontsize': 10,
        'lines.linewidth': 2,
    })

fig, ax = plt.subplots(figsize=(6, 5))

markers = ['s', 'D', '*', 'p']

for marker, (label, df) in zip(markers, duty_cycle_dfs.items()):
    ax.plot(100*df["duty_cycle"], 100*df["error_rate"], label=label, marker=marker)

ax.set_xlabel("Duty cycle [%]")
ax.set_ylabel("Error rate [%]")
ax.grid(True)
ax.set_ylim(0, 25)
ax.set_xlim(0, 100)

ax.legend(bbox_to_anchor=(0.5, 1.3), loc="upper center", ncol=2)

fig.tight_layout()
fig.savefig('duty_cycle_comparison.png', dpi=300)
plt.show()